<a href="https://colab.research.google.com/github/ankitg-02/data_analysis/blob/main/consumer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
consumer_sales=pd.read_excel("/content/Sample-sales-data-excel.xls")

In [ ]:
count_conumer=0
for i in consumer_sales['Segment']:
  if i=='Consumer':
    count_conumer+=1
print("count of conumers=",count_conumer)

count of conumers= 5191


In [34]:
record=consumer_sales.groupby('Segment')
print(record.Category.get_group("Consumer").value_counts())


Office Supplies    3127
Furniture          1113
Technology          951
Name: Category, dtype: int64
